In [13]:
from src.Analysis import Analysis

import math
import pandas as pd
import os
import numpy as np

In [14]:
class Config:
    tracks = 100
    segments = 10
    seconds = 3
    rate = 22050
    data_dir = 'data'
    lib_dir = 'D:\\Musik\\Genres\\'
    genres = [e for e in os.listdir(lib_dir) if e not in ['Trash', 'Other', 'JumpUp', 
               'Hiphop','Disco', 'Rock', 'Blues', 'Classical', 'Country', 'Reggae', 'Jazz', 'Metal', 'Pop',
               'DnB', 
               'Crossbread', 'Neurofunk', 'Sambass', 'Jungle', 'Drumstep', 'Deep', 'Liquid', 'Minimal', 'Dancefloor',
               'JumpUp', 
               'Techno', 'Goa', 'PsyTrance', 
               'Hardtekk', 'Riddim', 'Breakcore', 'Dubstep']]
    sub_dir = os.path.join(data_dir, f'{tracks}x{segments}x{seconds}')
    seed = 42
config = Config()
print(config.genres)

['Anne - Anne_-_Marie_-_Alarm_Tsuki_Bootleg_Free_Download-298463967.wav', 'Anne - Marie - Alarm (Tsuki Bootleg)(Free Download).mp3', 'Shriek', 'UKay']


In [15]:
from keras.models import load_model
reconstructed_model = load_model(os.path.join(config.sub_dir, "model.h5"))

import joblib
loaded_scaler = joblib.load(os.path.join(config.sub_dir, "scaler.pkl"))  # load from disk

import pickle
with open(os.path.join(config.sub_dir, "labels.pkl"), "rb") as a_file:
    GENRES = pickle.load(a_file)

In [16]:
print(GENRES)

{0: 'Shriek', 1: 'UKay'}


In [17]:
OUTPUT_PATH = 'D:\\Musik\\Genres\\'
DIR_PATH = 'D:\\Musik\\Genres\\JumpUp\\'
DIR_ERROR_PATH = DIR_PATH

In [18]:
files = os.listdir(DIR_PATH)

for file in files:
    if(os.path.splitext(file)[-1] not in ['.wav', '.mp3']):
        print('not supported')
        continue
    file_path = os.path.join(DIR_PATH, file)

    try:
        sis = Analysis(audio_path=file_path, options=config)

        inside_df = pd.DataFrame(sis.calculate())
        inside_df.drop(['length'], axis=1, inplace=True)
        inside_df = pd.DataFrame(loaded_scaler.transform(inside_df), columns=inside_df.columns)
        inside_df.reset_index(drop=True)

        q = reconstructed_model.predict(inside_df).T

    except Exception:
        continue

    points = [np.mean(r, axis=None) for r in q]
    biggest_number = max(points)

    guess = 'other'
    guess_save = 0
    alt_guess = 'other'
    
    temp = 0
    for i, r in enumerate(q):
        _max = np.max(r)
        temp += _max
        g = np.mean(r) * (1 - (np.var(r)))
        
        if((_max < 0.80)):
            continue
        if(g > guess_save):
            alt_guess = guess
            alt_guess_save = guess_save
            guess_save = g
            guess = GENRES[i]
        elif(g > alt_guess_save):
            alt_guess = GENRES[i]
            alt_guess_save = g

    # print('Avage: ', temp - guess_save)

    path_dir = os.path.join(OUTPUT_PATH, guess)
    path_file = os.path.join(DIR_PATH, file)
    new_file = os.path.join(path_dir, os.path.basename(file))
    
    if(guess_save < 0.8):
        continue

    if alt_guess != 'other':    
        print(f'{os.path.basename(path_file)} >> ({guess}: {round(np.max(guess_save)*100, 2)}% or {alt_guess}: {round(np.max(alt_guess_save)*100, 2)}%)')
    else:
        print(f'{os.path.basename(path_file)} >> ({guess}: {round(np.max(guess_save)*100, 2)}%)')

    # if not os.path.exists(OUTPUT_PATH):
    #     os.makedirs(OUTPUT_PATH)
    # if not os.path.exists(path_dir):
    #     os.makedirs(path_dir)
    
    # print(path_file, new_file)
    os.replace(path_file, new_file)

'Rick and Morty' - Theme (DnB⁄Hardtech)_468935403 - Sergeant Hardcore.mp3 >> (UKay: 87.07%)
02 - TC - HMRC_537505824 - tcdnb.mp3 >> (UKay: 99.15%)
02. Hedex - Proxic.mp3 >> (Shriek: 100.0%)
03 RedPill - Funk The Police (Eatbrain040)_3.mp3 >> (UKay: 99.97%)
03 RedPill - Funk The Police (Eatbrain040)_338404852 - Redpill.mp3 >> (UKay: 99.97%)
07 - TC - The Teacher_537505941 - tcdnb.mp3 >> (Shriek: 99.99%)
07. Hedex & Captain Bass ft. MC Skywalker - Soldiers.mp3 >> (UKay: 99.98%)
08. Hedex & Used - You Got Me.mp3 >> (UKay: 100.0%)
10AD & KREEPA - KREEPA RIDDIM (FREE DOWNLOAD)_846866587 - OnlyDrums.mp3 >> (Shriek: 93.55%)
14. Hedex - End Of The World.mp3 >> (UKay: 100.0%)
2019].mp3 >> (UKay: 100.0%)
2dB - Leave [Friction Premiere]_339299287 - TechniqueRecordings.mp3 >> (UKay: 99.15%)
3RDKND - The Artifact (Audio Remix) [Bassrush Premiere]_519316188 - Bassrush.mp3 >> (UKay: 100.0%)
50 Carrot & Coffi & Soloman - New President (Hedex VIP Remix)_247357828 - Thats Skanky!.mp3 >> (UKay: 100.0%)
5